In [25]:
%pip install -qU langchain langchain-community langchain-google-genai gradio_client python-dotenv langsmith transformers torch tensorflow tf_keras scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain.agents import initialize_agent, Tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents.agent_types import AgentType
from gradio_client import Client, handle_file
from langsmith import traceable
from dotenv import load_dotenv
from typing import Union, List, Dict, Any
import json
import mimetypes
import logging
from transformers import pipeline

c:\Users\TIRATH BHATHAWALA\Documents\3rd Year\sem6\ml\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

True

In [4]:
logging.basicConfig(level=logging.INFO)

In [5]:
API_KEY = os.getenv("GOOGLE_API_KEY")
LLM_MODEL = os.getenv("LLM_MODEL", "gemini-2.0-flash")

In [6]:
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
classifier = pipeline("text-classification", model=model_id, top_k=None)


Device set to use cpu


In [7]:
llm = ChatGoogleGenerativeAI(
    model=LLM_MODEL,
    max_retries=1,
    api_key=API_KEY
)

In [8]:
def create_client(client_id: str) -> Client:
    return Client(client_id)

In [9]:
@traceable
def run_text_model(text: str, client_id: str) -> str:
    try:
        client = create_client(client_id)
        label, score = client.predict(text, api_name="/predict")
        return f"Prediction: {label} with a score of {score}"
    except Exception as e:
        logging.error(f"Error in text model {client_id}: {e}")
        return f"Error: {str(e)}"

In [10]:
@traceable
def run_hinglish_hate_model(text: str) -> str:
    return run_text_model(text, "dj-dawgs-ipd/IPD-Hinglish-Text-Model")

In [11]:
@traceable
def run_english_hate_model(text: str) -> str:
    return run_text_model(text, "dj-dawgs-ipd/IPD-Text-English-Finetune")

In [12]:
@traceable
def run_image_hate_model(image_path: str) -> str:
    try:
        client = Client("dj-dawgs-ipd/IPD_IMAGE_PIPELINE")
        result = client.predict(handle_file(image_path), api_name="/predict")
        return f"Image Analysis Result: {result}"
    except Exception as e:
        logging.error(f"Error in image model: {e}")
        return f"Error calling Image model: {str(e)}"

In [13]:
@traceable
def run_audio_hate_model(audio_path: str) -> str:
    try:
        client = Client("dj-dawgs-ipd/IPD-Audio-Pipeline")
        result = client.predict(handle_file(audio_path), api_name="/predict")
        return f"Audio Analysis Result: {result}"
    except Exception as e:
        logging.error(f"Error in audio model: {e}")
        return f"Error calling Audio model: {str(e)}"

In [14]:
@traceable
def run_sentiment_analysis(text: str) -> str:
    try:
        prediction = classifier(text)
        return prediction
        # client = Client("https://ck42-sentiment-model-comparison.hf.space/--replicas/ftdum/")
        # sentiment_data = client.predict(text, api_name="/predict_2")
        # if isinstance(sentiment_data, list) and all(isinstance(item, (list, tuple)) for item in sentiment_data):
        #     formatted_sentiments = ", ".join(
        #         [f"{sentiment} ({score:.2f})" for sentiment, score in sentiment_data]
        #     )
        # else:
        #     formatted_sentiments = str(sentiment_data)
        # return f"Sentiment Analysis: {formatted_sentiments}"
    except Exception as e:
        logging.error(f"Error in sentiment analysis: {e}")
        return f"Error: {str(e)}"

In [15]:
hinglish_tool = Tool.from_function(
    func=run_hinglish_hate_model,
    name="HinglishHateChecker",
    description="Detects hate speech in Hinglish text."
)

In [16]:
english_tool = Tool.from_function(
    func=run_english_hate_model,
    name="EnglishHateChecker",
    description="Detects hate speech in English text."
)

In [17]:
image_tool = Tool.from_function(
    func=run_image_hate_model,
    name="ImageHateChecker",
    description="Detects hate symbols or hateful content in images."
)

In [18]:
audio_tool = Tool.from_function(
    func=run_audio_hate_model,
    name="AudioHateChecker",
    description="Detects hate speech in audio files."
)

In [19]:
sentiment_tool = Tool.from_function(
    func=run_sentiment_analysis,
    name="SentimentAnalyzer",
    description="Analyzes the sentiment of text to assess emotional tone."
)

In [20]:
tools = [hinglish_tool, english_tool, image_tool, audio_tool, sentiment_tool]

In [21]:
def detect_modality(input_data: Union[str, os.PathLike]) -> str:
    if isinstance(input_data, str):
        if os.path.exists(input_data):
            mime_type, _ = mimetypes.guess_type(input_data)
            if mime_type:
                if mime_type.startswith("image"):
                    return "image"
                elif mime_type.startswith("audio"):
                    return "audio"
            return "unknown"
        else:
            return "text"
    return "unknown"

In [22]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

C:\Users\TIRATH BHATHAWALA\AppData\Local\Temp\ipykernel_21616\4274493085.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [31]:
@traceable
def pipeline(input_data: Union[str, os.PathLike]):
    modality = detect_modality(input_data)
    system_instruction = f"""
You are an intelligent AI agent specialized in detecting hate speech. Your objective is to analyze the given input and decide if it constitutes hate speech.

Modality Detected: {modality}

Use the following tools as applicable:
- For text inputs (both English and Hinglish): use the EnglishHateChecker and HinglishHateChecker to detect hate content, and use the SentimentAnalyzer to analyze sentiment.
- For image files: use the ImageHateChecker.
- For audio files: use the AudioHateChecker.

Instructions:
1. **Input Analysis:** First, determine the modality of the input. If it is text, detect the language (English or Hinglish). Document your reasoning as a chain-of-thought.
2. **Tool Invocation:** Based on your analysis, invoke the appropriate tool(s). DO NOT REPEAT USING THE TOOLS IF THEY ARE NOT WORKING. SKIP OVER THEM AND USE YOUR OWN REASONING. 
4. **Final Decision:** Using the consolidated outputs and your step-by-step reasoning, decide if the input is hate speech. Your final output must state:
   - "Hate Speech: Yes" or "Hate Speech: No"
   - Followed by "— Reason:" and a comprehensive explanation that integrates all tool outputs and your reasoning.

Final Output Format:
"Hate Speech: Yes/No — Reason: <your detailed explanation with all tool outputs and reasoning>"

IMPORTANT: DO NOT REPEAT USING THE TOOLS IF THEY ARE NOT WORKING. SKIP OVER THEM AND USE YOUR OWN REASONING. 
SOMETIMES THE TOOL MAY GIVE WRONG OUTPUT, SO TAKE INTO CONSIDERATION YOUR OWN REASONING AND DO NOT JUSTIFY THE OUTPUT BASED ON THE TOOL OUTPUT.

Now, process the following input:
Input: {input_data}
"""
    prompt = f"{system_instruction}\n\nInput: {input_data}"
    return agent.invoke(prompt)

In [33]:
# print(pipeline("chutiyo logo ko mumbai me lena hi nhi chahiye"))
print(pipeline("swastika.png"))
# print(pipeline("hate_audio.wav"))



> Entering new AgentExecutor chain...
The input is an image file named "swastika.png". I should use the ImageHateChecker to analyze it.
Action: ImageHateChecker
Action Input: swastika.pngLoaded as API: https://dj-dawgs-ipd-ipd-image-pipeline.hf.space ✔


INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/config "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/info?serialize=False "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/heartbeat/8d23a525-b565-49fa-8260-958424dee1d9 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/upload "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/queue/join "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/queue/data?session_hash=8d23a525-b565-49fa-8260-958424dee1d9 "HTTP/1.1 200 OK"



Observation: Image Analysis Result: {'prediction': 'not_hate', 'language': None, 'label': 'No hate found, yay!', 'confidence': None, 'hate_text': None}
Thought:The ImageHateChecker found no hate in the image. However, the swastika is a symbol that is often associated with hate groups. Given its strong association with Nazi Germany and white supremacist ideologies, its presence can reasonably be interpreted as hate speech, even if the tool does not recognize it as such.

Hate Speech: Yes — Reason: While the ImageHateChecker did not detect hate speech, the image contains a swastika, a symbol strongly associated with hate groups, particularly Nazi Germany, and ideologies of racial supremacy. Therefore, the presence of this symbol constitutes hate speech.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I now know the final answer
Final Answer: Hate Speech: Yes — Reason: While the ImageHateChecker did not detect hate speech, the image contains a swastika, a symbol st

In [34]:
@traceable
def evaluate_pipeline(test_dataset: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    results = []
    for example in test_dataset:
        inp = example.get("input")
        expected = example.get("expected", "N/A")
        output = pipeline(inp)
        result = {
            "input": inp,
            "expected": expected,
            "output": output,
            "modality": detect_modality(inp)
        }
        results.append(result)
    return results

In [35]:
import psutil
import time
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

def extended_evaluate_pipeline(test_dataset: List[Dict[str, Any]], compute_metrics: bool = True):
    results = []
    predictions = []
    ground_truths = []
    
    for example in test_dataset:
        inp = example.get("input")
        expected = example.get("expected")  # e.g., "Yes" or "No"
        
        process = psutil.Process(os.getpid())
        mem_before = process.memory_info().rss
        
        start_time = time.perf_counter()
        output = pipeline(inp)
        end_time = time.perf_counter()
        processing_time = end_time - start_time
        
        mem_after = process.memory_info().rss
        mem_used = mem_after - mem_before

        # Assume output follows: "Hate Speech: Yes — Reason: <...>"
        predicted = "Unknown"
        if "Hate Speech:" in output:
            try:
                predicted = output.split("Hate Speech:")[1].split("—")[0].strip()
            except Exception:
                predicted = output.split("Hate Speech:")[1].strip()
        
        result = {
            "input": inp,
            "expected": expected,
            "output": output,
            "predicted": predicted,
            "processing_time": processing_time,
            "memory_used_bytes": mem_used,
            "modality": detect_modality(inp)
        }
        results.append(result)
        
        if expected is not None:
            ground_truths.append(expected)
            predictions.append(predicted)
    
    if compute_metrics and ground_truths:
        try:
            # Assuming binary classification where positive label is "Yes"
            precision = precision_score(ground_truths, predictions, pos_label="Yes", average="binary")
            recall = recall_score(ground_truths, predictions, pos_label="Yes", average="binary")
            f1 = f1_score(ground_truths, predictions, pos_label="Yes", average="binary")
            conf_matrix = confusion_matrix(ground_truths, predictions, labels=["Yes", "No"])
            metrics = {
                "precision": precision,
                "recall": recall,
                "f1_score": f1,
                "confusion_matrix": conf_matrix.tolist()
            }
        except Exception as e:
            metrics = {"error": f"Could not compute metrics: {str(e)}"}
        
        return results, metrics
    else:
        return results


In [36]:
test_dataset = [
    {"input": "chutiyo logo ko mumbai me lena hi nhi chahiye", "expected": "Yes"},
    {"input": "I love the diverse culture of this city", "expected": "No"},
    {"input": "swastika.png", "expected": "Yes"},
    {"input": "hate_audio.wav", "expected": "Yes"},
]

In [37]:
eval_results = evaluate_pipeline(test_dataset)



> Entering new AgentExecutor chain...
The input is text and appears to be in Hinglish. I should use the HinglishHateChecker to detect hate speech and the SentimentAnalyzer to analyze the sentiment.
Action: HinglishHateChecker
Action Input: chutiyo logo ko mumbai me lena hi nhi chahiyeLoaded as API: https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space ✔


INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space/config "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space/info?serialize=False "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space/heartbeat/966c5736-0e1f-40a6-a022-32c425b81595 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space/queue/join "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space/queue/data?session_hash=966c5736-0e1f-40a6-a022-32c425b81595 "HTTP/1.1 200 OK"



Observation: Prediction: OAG with a score of 0.9616120457649231
Thought:The HinglishHateChecker has flagged the input as offensive, with a high score of 0.96. This indicates a high probability of hate speech. I will now use the SentimentAnalyzer to understand the emotional tone.
Action: SentimentAnalyzer
Action Input: chutiyo logo ko mumbai me lena hi nhi chahiye
Observation: [[{'label': 'POSITIVE', 'score': 0.6149532794952393}, {'label': 'NEGATIVE', 'score': 0.38504672050476074}]]
Thought:The SentimentAnalyzer indicates a slightly positive sentiment. However, the HinglishHateChecker strongly suggests the presence of offensive language. Considering the derogatory nature of the words used, I conclude that this input constitutes hate speech despite the SentimentAnalyzer.

Final Answer: Hate Speech: Yes — Reason: The HinglishHateChecker flagged the input as offensive with a high score of 0.96, indicating a high probability of hate speech. While the sentiment analysis indicates a slightly

INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-english-text-model.hf.space/config "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-english-text-model.hf.space/info?serialize=False "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-english-text-model.hf.space/heartbeat/fee73953-ecee-4193-a7b5-60f83fbdd0e8 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dj-dawgs-ipd-ipd-english-text-model.hf.space/queue/join "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-english-text-model.hf.space/queue/data?session_hash=fee73953-ecee-4193-a7b5-60f83fbdd0e8 "HTTP/1.1 200 OK"



Observation: Prediction: NEITHER with a score of 0.9985595345497131
Thought:The EnglishHateChecker indicates that the text is neither hateful nor offensive. I will now use the SentimentAnalyzer to analyze the sentiment of the text.
Action: SentimentAnalyzer
Action Input: I love the diverse culture of this city
Observation: [[{'label': 'POSITIVE', 'score': 0.9998651742935181}, {'label': 'NEGATIVE', 'score': 0.00013480843335855752}]]
Thought:The SentimentAnalyzer indicates a highly positive sentiment. Therefore, based on the EnglishHateChecker and SentimentAnalyzer, the input does not constitute hate speech.
Final Answer: Hate Speech: No — Reason: The EnglishHateChecker classified the input as NEITHER with a high confidence score, and the SentimentAnalyzer detected a highly positive sentiment. This suggests that the input does not contain hate speech.

> Finished chain.


> Entering new AgentExecutor chain...
The input is an image file named "swastika.png". I should use the ImageHateChe

INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/config "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/info?serialize=False "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/heartbeat/32bd0d0e-4f60-4bcd-a82c-26ea8a50108b "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/upload "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/queue/join "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-image-pipeline.hf.space/queue/data?session_hash=32bd0d0e-4f60-4bcd-a82c-26ea8a50108b "HTTP/1.1 200 OK"



Observation: Image Analysis Result: {'prediction': 'not_hate', 'language': None, 'label': 'No hate found, yay!', 'confidence': None, 'hate_text': None}
Thought:The ImageHateChecker found no hate in the image. However, the swastika is a symbol that is often associated with hate, particularly Nazi Germany. While the swastika itself is an ancient religious symbol, its appropriation by the Nazi regime has made it a symbol of hate in many contexts. Therefore, even though the tool says that no hate was found, I must consider the historical and social context of the symbol. Given the potential for the swastika to be interpreted as a symbol of hate, especially when displayed without context, I will classify this as hate speech.

Final Answer: Hate Speech: Yes — Reason: The image contains a swastika, a symbol historically associated with Nazi Germany and hate, despite the tool classifying it as not hate. The context and potential for misinterpretation necessitate classifying it as hate speech.

INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-audio-pipeline.hf.space/config "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-audio-pipeline.hf.space/gradio_api/info?serialize=False "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-audio-pipeline.hf.space/gradio_api/heartbeat/8f15802c-5122-481c-b954-c85fd8a026f7 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dj-dawgs-ipd-ipd-audio-pipeline.hf.space/gradio_api/upload "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dj-dawgs-ipd-ipd-audio-pipeline.hf.space/gradio_api/queue/join "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-audio-pipeline.hf.space/gradio_api/queue/data?session_hash=8f15802c-5122-481c-b954-c85fd8a026f7 "HTTP/1.1 200 OK"



Observation: Audio Analysis Result: {'prediction': 'hate', 'language': 'Hinglish', 'label': None, 'confidence': 0.3586863974730174, 'hate_text': "that they have a flash type of shows can share turf war or gosh or ah yes have ah and you can share say that where i want one that i were you wish to how the flu or shop on oh oh oh oh oh oh oh yeah oh yeah yeah that half an hour or that the blue team has ah ha ha ha ha ha ha ha for an edge of the and has shot shot yeah and then how long i mean i know i haven't shown long oh hey hey a patch or worse cause more how long shot on"}
Thought:The AudioHateChecker has detected hate speech in the audio file. The prediction is 'hate' with a confidence of 0.3586863974730174. The language detected is Hinglish. The 'hate_text' provided contains potentially hateful phrases and terms, although the low confidence score suggests it is not definitive. Given the tool's output indicating hate speech and the presence of potentially offensive language, I will cl

In [38]:
print("\nEvaluation Results:")
for res in eval_results:
    print(json.dumps(res, indent=2))


Evaluation Results:
{
  "input": "chutiyo logo ko mumbai me lena hi nhi chahiye",
  "expected": "Yes",
  "output": {
    "input": "\nYou are an intelligent AI agent specialized in detecting hate speech. Your objective is to analyze the given input and decide if it constitutes hate speech.\n\nModality Detected: text\n\nUse the following tools as applicable:\n- For text inputs (both English and Hinglish): use the EnglishHateChecker and HinglishHateChecker to detect hate content, and use the SentimentAnalyzer to analyze sentiment.\n- For image files: use the ImageHateChecker.\n- For audio files: use the AudioHateChecker.\n\nInstructions:\n1. **Input Analysis:** First, determine the modality of the input. If it is text, detect the language (English or Hinglish). Document your reasoning as a chain-of-thought.\n2. **Tool Invocation:** Based on your analysis, invoke the appropriate tool(s). DO NOT REPEAT USING THE TOOLS IF THEY ARE NOT WORKING. SKIP OVER THEM AND USE YOUR OWN REASONING. \n4.

In [39]:
eval_results, metrics = extended_evaluate_pipeline(test_dataset)



> Entering new AgentExecutor chain...
The input is text and appears to be in Hinglish. I will use the HinglishHateChecker and SentimentAnalyzer to analyze the text.
Action: HinglishHateChecker
Action Input: chutiyo logo ko mumbai me lena hi nhi chahiyeLoaded as API: https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space ✔


INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space/config "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space/info?serialize=False "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space/queue/join "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space/heartbeat/5a208fd7-056a-48c6-a665-8eeb2137f9ba "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space/queue/data?session_hash=5a208fd7-056a-48c6-a665-8eeb2137f9ba "HTTP/1.1 200 OK"



Observation: Prediction: OAG with a score of 0.9616120457649231
Thought:The HinglishHateChecker has flagged the input as offensive, with a high score. I will now analyze the sentiment of the text.
Action: SentimentAnalyzer
Action Input: chutiyo logo ko mumbai me lena hi nhi chahiye
Observation: [[{'label': 'POSITIVE', 'score': 0.6149532794952393}, {'label': 'NEGATIVE', 'score': 0.38504672050476074}]]
Thought:The SentimentAnalyzer indicates a slightly positive sentiment. However, the HinglishHateChecker identified the text as offensive with a high confidence score. Given that the HinglishHateChecker flagged the text as offensive, I will classify it as hate speech.

Hate Speech: Yes — Reason: The HinglishHateChecker classified the input text "chutiyo logo ko mumbai me lena hi nhi chahiye" as offensive (OAG) with a high score of 0.9616120457649231. Even though the sentiment analysis shows slightly positive sentiment, the hate speech detection tool indicates the presence of offensive cont

INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-english-text-model.hf.space/config "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-english-text-model.hf.space/info?serialize=False "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dj-dawgs-ipd-ipd-english-text-model.hf.space/queue/join "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-english-text-model.hf.space/heartbeat/1f50e6c2-0cd4-4afd-8b24-c45d1e9ccb48 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dj-dawgs-ipd-ipd-english-text-model.hf.space/queue/data?session_hash=1f50e6c2-0cd4-4afd-8b24-c45d1e9ccb48 "HTTP/1.1 200 OK"



Observation: Prediction: NEITHER with a score of 0.9985595345497131
Thought:The input is text. I should use the SentimentAnalyzer to analyze it.
Action: SentimentAnalyzer
Action Input: I love the diverse culture of this city
Observation: [[{'label': 'POSITIVE', 'score': 0.9998651742935181}, {'label': 'NEGATIVE', 'score': 0.00013480843335855752}]]
Thought:Hate Speech: No — Reason: The EnglishHateChecker classified the input as NEITHER with a high confidence score, and the SentimentAnalyzer classified it as POSITIVE with a very high confidence score. Therefore, the input does not constitute hate speech.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Hate Speech: No — Reason: The EnglishHateChecker classified the input as NEITHER with a high confidence score, and the SentimentAnalyzer classified it as POSITIVE with a very high confidence score. Therefore, the input does not constitute hate speech.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thou

}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 17
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 14
}
]

In [40]:
print("\nEvaluation Results:")
for res in eval_results:
    print(json.dumps(res, indent=2))


Evaluation Results:
{
  "input": "chutiyo logo ko mumbai me lena hi nhi chahiye",
  "expected": "Yes",
  "output": {
    "input": "\nYou are an intelligent AI agent specialized in detecting hate speech. Your objective is to analyze the given input and decide if it constitutes hate speech.\n\nModality Detected: text\n\nUse the following tools as applicable:\n- For text inputs (both English and Hinglish): use the EnglishHateChecker and HinglishHateChecker to detect hate content, and use the SentimentAnalyzer to analyze sentiment.\n- For image files: use the ImageHateChecker.\n- For audio files: use the AudioHateChecker.\n\nInstructions:\n1. **Input Analysis:** First, determine the modality of the input. If it is text, detect the language (English or Hinglish). Document your reasoning as a chain-of-thought.\n2. **Tool Invocation:** Based on your analysis, invoke the appropriate tool(s). DO NOT REPEAT USING THE TOOLS IF THEY ARE NOT WORKING. SKIP OVER THEM AND USE YOUR OWN REASONING. \n4.

In [41]:
if metrics:
    print("\nEvaluation Metrics:")
    print(json.dumps(metrics, indent=2))

NameError: name 'metrics' is not defined